In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train_clean.csv')
df.head(3)

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,0,15674932,Okwudilichukwu,668,France,1,33.0,3,0.0,2,1.0,0.0,181449.97,0,1,0,0
1,1,15749177,Okwudiliolisa,627,France,1,33.0,1,0.0,2,1.0,1.0,49503.50,0,1,0,0
2,2,15694510,Hsueh,678,France,1,40.0,10,0.0,2,1.0,0.0,184866.69,0,1,0,0


## Modelado

### Feature Engineering

In [5]:
#NORMALIZACION
from sklearn.preprocessing import StandardScaler

#LISTA DE VARIABLES A NORMALIZAR
vars = ['CreditScore','Age','Balance','EstimatedSalary']

#INSTANCIA DEL MODELO Y TRANSOFRMACION
std_vars = StandardScaler().fit_transform(df[vars])

#SE CREA DATAFRAME COMPLEMETARIO
std_vars = pd.DataFrame(std_vars, columns=vars)
std_vars.head(3)

,CreditScore,Age,Balance,EstimatedSalary
0,0.144135,-0.576143,-0.883163,1.369486
1,-0.367706,-0.576143,-0.883163,-1.254085
2,0.268974,0.335336,-0.883163,1.437422


In [7]:
#SELECCION DE VARIABLES
vars = ['France','Germany','Spain','NumOfProducts','HasCrCard','IsActiveMember']

#VARIABLES PREDICTIVAS
x = pd.concat([df[vars], std_vars], axis=1)

#VARIABLE OBJETIVO
y = df.Exited